In [20]:
from tweepy import OAuthHandler 
from tweepy import API
import json

# obtain free API key: 
# https://developer.twitter.com/en/docs/basics/authentication/guides/access-tokens.html
consumer_key = "SUuqUxf2K3hiPSUCcUSwvkVlu"
consumer_secret = "KHGoQG1BBnhhI2xoZQLoFo9MnlrvMbdg0LGo7dD1hB8NMSvizf"
access_token = "1078941783306846208-G7jZdgFsnTXT9kwlFRimO5e4AYAYxc"
access_token_secret = "eDgEqYNSijBKYIHZcraq73x43WzTYiVVnZRVKMgHpdP6T"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = API(auth)
# we can query this api object for data from twitter

tweets = api.user_timeline(screen_name='@realDonaldTrump',count=5) 

# limited to 200 per request
for i in range(len(tweets)):
    print(tweets[i].text)
    print()

I will be interviewed by @seanhannity at 9:00 P.M. on @FoxNews. Enjoy!

Everyone MUST watch the opening to @TuckerCarlson - A Classic! @foxnews

THANK YOU to our generous hosts in Hanoi this week: President Trong, Prime Minister Phuc, and the wonderful people… https://t.co/dYi5WHA50b

RT @StateDept: President @realDonaldTrump and @SecPompeo participate in a Press Conference in Hanoi, Vietnam, on February 28, 2019. https:/…

RT @realDonaldTrump: Michael Cohen was one of many lawyers who represented me (unfortunately). He had other clients also. He was just disba…



['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_max_id',
 '_since_id',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'ids',
 'index',
 'insert',
 'max_id',
 'pop',
 'remove',
 'reverse',
 'since_id',
 'sort']

Twitter provides up to 200 tweets per query via this method up to a total of the user's 3240 most recent tweets. @realDonaldTrump has tweeted much more than this: as of January 2019, 3240 tweets represents approximately a 10-month history.

In [2]:
print(api.get_user(screen_name='@realDonaldTrump').statuses_count) 
# total since account creation in May 2009

40689


Since the API is too restrictive to collect a full history, let's load a json file from a third-party archive:
http://www.trumptwitterarchive.com/archive

In [3]:
import pandas as pd

alltweets_df = pd.read_json('theTorrent.json') # create a DataFrame object of all tweets
# source: http://www.trumptwitterarchive.com/archive
# retrieved Jan 9 

alltweets_df.head(5)

,created_at,favorite_count,id_str,is_retweet,retweet_count,source,text
0,2019-01-09 20:34:46,191731,1083099784012091392,0.0,45931,Twitter for iPad,"Just left a meeting with Chuck and Nancy, a to..."
1,2019-01-09 19:31:29,89540,1083083858357248000,0.0,21900,Twitter for iPhone,"Thank you to all of America's brave police, de..."
2,2019-01-09 15:25:43,104118,1083022011574747136,0.0,24088,Twitter for iPhone,Billions of dollars are sent to the State of C...
3,2019-01-09 14:36:05,30626,1083009519503003648,0.0,7635,Twitter for iPhone,Billions of dollars are sent to the State of C...
4,2019-01-09 14:06:09,137562,1083001986331672576,0.0,29217,Twitter for iPhone,Our Country is doing so well in so many ways. ...


In [4]:
alltweets_df.shape

(36443, 7)

Here is a quick script to count how often a subject is tweeted about.

In [5]:
def count_tweets(search_str):
    count = 0
    for index, row in alltweets_df.iterrows():
        if search_str in row['text'].lower():
            count += 1
        
    return count
# end

count_tweets('fake news')

346

Let's use this script to count mentions of a few countries.

In [6]:
count_tweets('canada')

101

In [7]:
count_tweets('mexico')

201

In [8]:
count_tweets('russia')

415

In [9]:
count_tweets('china')

551

In [10]:
count_tweets('iran')

308

In [11]:
count_tweets('iraq')

170

In [12]:
count_tweets('sweden') # both re: the terrorist attack he invented there

2

The script below populates a dictionary with each unique word then sorts the result by frequency.

In [13]:
import re

def the_best_words(alltweets_df):
    
    count = 0
    all_count = 0
    words = {}
    
    for i in range(len(alltweets_df)):
        for word in alltweets_df['text'].loc[i].lower().split(' '):
            all_count += 1

            regex = re.compile('[^a-zA-Z]')
            word = regex.sub('', word) # remove all non-alphabetic chars

            if word in words: # 
                words[word] += 1
            else:
                words[word] = 1
                count += 1
        
    words_df = pd.DataFrame.from_dict(words, orient='index', columns=['count'])
    words_df = words_df.sort_values('count', ascending=False)
    # remove some common words
    remove_words = ['', 'the', 'to', 'a', 'and', 'is', 'of', 'in', 'you', 'for', 'i', 'on', 'be', 'will', 'it', 'are',
                   'that', 'with', 'we', 'have', 'at', 'amp', 'our', 'my', 'he', 'this', 'not', 'was', 'all', 'your',
                    'by', 'has', 'they', 'so', 'just', 'as', 'us', 'from', 'who', 'do', 'his', 'what', 'if', 'no',
                    'get', 'but', 'would', 'about', 'an', 'dont', 'its']
    
    words_df = words_df.drop(remove_words)

    return words_df

In [14]:
words_df = the_best_words(alltweets_df)
words_df.head(30)

,count
realdonaldtrump,8685
trump,5585
great,5108
thank,2335
president,2306
thanks,2023
people,1967
donald,1727
very,1676
me,1660
